### Connect to cluster

In [10]:
import ray
import ray_tpu

ray.init()

2025-04-15 03:27:33,295	INFO worker.py:1660 -- Connecting to existing Ray cluster at address: 10.128.0.32:6379...
2025-04-15 03:27:33,302	INFO worker.py:1843 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.14
Ray version:,2.44.1
Dashboard:,http://127.0.0.1:8265


(run_on_tpu pid=3448310, ip=10.138.0.2) WARNING:root:libtpu.so and TPU device found. Setting PJRT_DEVICE=TPU.


(pid=, ip=10.138.0.2) Log file created at: 2025/04/14 20:28:33
(pid=, ip=10.138.0.2) Running on machine: t1v-n-8ed780ed-w-0
(pid=, ip=10.138.0.2) Binary: Built on Jan 24 2025 17:49:30 (1737769770)
(pid=, ip=10.138.0.2) Binary: Built at cloud-tpus-runtime-release-tool@lmbco37.prod.google.com:/google/src/cloud/buildrabbit-username/buildrabbit-client/g3     
(pid=, ip=10.138.0.2) Binary: Built for gcc-4.X.Y-crosstool-v18-llvm-grtev4-k8.k8
(pid=, ip=10.138.0.2) Log line format: [IWEF]mmdd hh:mm:ss.uuuuuu threadid file:line] msg
(pid=, ip=10.138.0.2) I0414 20:28:33.657100 3448310 b295d63588a.cc:733] Linux version 6.5.0-1013-gcp (buildd@lcy02-amd64-064) (x86_64-linux-gnu-gcc-12 (Ubuntu 12.3.0-1ubuntu1~22.04) 12.3.0, GNU ld (GNU Binutils for Ubuntu) 2.38) #13~22.04.1-Ubuntu SMP Wed Jan 24 23:39:40 UTC 2024
(pid=, ip=10.138.0.2) I0414 20:28:33.667502 3448310 b295d63588a.cc:815] Process id 3448310
(pid=, ip=10.138.0.2) I0414 20:28:33.667528 3448310 b295d63588a.cc:820] Current working directory 

In [11]:
ray.available_resources()

{'GPU': 1.0,
 'accelerator_type:A100': 1.0,
 'memory': 399361834190.0,
 'CPU': 236.0,
 'node:10.128.0.32': 1.0,
 'object_store_memory': 171155071794.0,
 'node:__internal_head__': 1.0,
 'TPU': 8.0,
 'yifeit-v5e': 1.0,
 'node:10.138.0.2': 1.0,
 'accelerator_type:TPU-V5LITEPOD': 1.0,
 'TPU-v5litepod-8-head': 1.0}

### Run some computation

In [25]:
@ray.remote(num_gpus=1)
def run_on_gpu(data):
  import torch
  return torch.cos(torch.sin(torch.tensor(data).cuda())).cpu().numpy()

@ray.remote(
    resources={"TPU": 1}
)
def run_on_tpu(data):
  import torch
  import torch_xla
  return torch.cos(torch.sin(torch.tensor(data).to('xla'))).cpu().numpy()

In [26]:
import numpy as np

input_data = np.random.rand(1, 224, 224, 3).astype(np.float32) # Example shape
input_data_ref = ray.put(input_data) # Send data to cluster object store

In [27]:
gpu_future = run_on_gpu.remote(input_data_ref)

In [28]:
ray.get(gpu_future).shape

(1, 224, 224, 3)

In [29]:
tpu_future = run_on_tpu.remote(input_data_ref)

In [30]:
ray.get(tpu_future).shape  # type: ignore

(1, 224, 224, 3)

In [31]:
import torch

torch.testing.assert_close(
    ray.get(tpu_future),
    ray.get(gpu_future),
)

In [32]:
ray.shutdown()